In [1]:
# Import dependencies
import pandas as pd
import requests
import gmaps
import gmaps.datasets
import numpy as np

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'gmaps'

In [2]:
# Read data from csv
filepath = 'Resources/Real-Time_Traffic_Incident_Reports.csv'
df = pd.read_csv(filepath)

In [3]:
# View
df.head()

,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status,Status Date
0,C163BCD1CF90C984E9EDA4DBA311BCA369A7D1A1_15288...,06/13/2018 06:35:59 AM +0000,Crash Service,"(30.283797,-97.741906)",30.283797,-97.741906,W 21ST ST & GUADALUPE ST,ARCHIVED,06/13/2018 09:00:03 AM +0000
1,6B20382196FB454E9FD06A33E60142902A2F0706_15288...,06/13/2018 10:15:36 AM +0000,Traffic Hazard,"(30.339593,-97.700963)",30.339593,-97.700963,400-717 E ANDERSON LN EB,ARCHIVED,06/13/2018 11:20:03 AM +0000
2,2A7A84FC69D56A2C379C274D2F0831442D3E7B8E_15871...,04/17/2020 09:25:03 PM +0000,Crash Urgent,"(30.329455,-97.638105)",30.329455,-97.638105,E US 290 HWY SVRD EB & EASTERN HEIGHTS BLVD,ARCHIVED,04/17/2020 09:55:03 PM +0000
3,89D3B75BB5932F10D25B5852A9902DA26C3DF11A_15871...,04/17/2020 09:40:52 PM +0000,Traffic Hazard,"(30.202806,-97.760701)",30.202806,-97.760701,5300-blk S Ih 35 Sb,ARCHIVED,04/17/2020 09:55:03 PM +0000
4,E7F9EFF0E81057A5C8FE41CABE4D9C6EEA82DAA0_15871...,04/17/2020 09:00:55 PM +0000,Crash Urgent,"(30.184265,-97.687339)",30.184265,-97.687339,4500-4720 S Us 183 Hwy Sb,ARCHIVED,04/17/2020 11:30:04 PM +0000


In [4]:
# Drop rows wth NaN values
df.dropna(axis=0, inplace=True)

In [5]:
# Convert dates to datetime format
df['Published Date'] = pd.to_datetime(df['Published Date'].str[:22], format='%m/%d/%Y %I:%M:%S %p')
df['Status Date'] = pd.to_datetime(df['Status Date'].str[:22], format='%m/%d/%Y %I:%M:%S %p')

In [6]:
pd.DataFrame(df['Issue Reported'].value_counts())

,Issue Reported
Traffic Hazard,101959
Crash Urgent,80605
Crash Service,47294
COLLISION,32448
TRFC HAZD/ DEBRIS,24563
LOOSE LIVESTOCK,9154
zSTALLED VEHICLE,8658
COLLISION WITH INJURY,8337
COLLISN/ LVNG SCN,6153
Stalled Vehicle,6125


In [7]:
df['Latitude'].sort_values(ascending=True)

313613    3.000303e+01
37452     3.000388e+01
54119     3.000441e+01
207587    3.000462e+01
255501    3.000480e+01
              ...     
307787    2.999245e+07
307997    2.999550e+07
20532     2.999586e+07
261577    2.999586e+07
232091    2.999603e+07
Name: Latitude, Length: 335253, dtype: float64

In [8]:
# Remove Latitude data out of range
df = df.loc[df['Latitude'].abs() < 90]

In [9]:
# Filter out dates
df_sample = df.loc[df['Published Date'] > pd.Timestamp('08/01/2023')]
df_sample

,Traffic Report ID,Published Date,Issue Reported,Location,Latitude,Longitude,Address,Status,Status Date
292907,9CB40392AAFD5B7A843D961C3628711D85FF5953_16917...,2023-08-10 20:52:05,Traffic Hazard,"(30.263939,-97.763747)",30.263939,-97.763747,1810 Barton Springs Rd,ARCHIVED,2023-08-10 22:20:09
292909,226E8783740869EA2C293750EC704DBCBD281B71_16917...,2023-08-10 21:55:49,COLLISION,"(30.349001,-97.546689)",30.349001,-97.546689,E Us 290 Hwy Eb / Shadowglen Blvd,ARCHIVED,2023-08-10 22:20:09
292910,E191806F057D4764D052354C3DC525868B91D707_16917...,2023-08-10 21:40:11,Traffic Hazard,"(30.218117,-97.692141)",30.218117,-97.692141,7200-7644 E Ben White Blvd Wb,ARCHIVED,2023-08-10 22:20:09
293197,A6571BB4231C92B5AF46954A0950FDEEFF69F384_16921...,2023-08-15 18:46:54,Crash Urgent,"(30.185623,-97.923055)",30.185623,-97.923055,7722-7737 Espina Dr,ARCHIVED,2023-08-15 20:25:03
335168,74BFAC88EC6B72E7C070A86C73A8F953FF99937D_16908...,2023-08-01 00:08:52,Traffic Hazard,"(30.167275,-97.785743)",30.167275,-97.785743,9000 S IH 35 NB,ARCHIVED,2023-08-01 00:25:03
...,...,...,...,...,...,...,...,...,...
337716,23A63310B8E95A3702B44B2F22F3248DE1561E88_16923...,2023-08-17 21:21:08,Crash Service,"(30.279185,-97.743734)",30.279185,-97.743734,San Antonio St / W 16th St,ACTIVE,2023-08-17 21:25:02
337717,6896DDC64EF4051E872B1E3B3B37D25E74F3C335_16923...,2023-08-17 21:21:30,Crash Service,"(30.234754,-97.723566)",30.234754,-97.723566,2512 E Riverside Dr,ACTIVE,2023-08-17 21:25:02
337718,08BC846FFF3A96EB1D36D184F604497CFF2F3727_16923...,2023-08-17 21:22:05,COLLISN/ LVNG SCN,"(30.345284,-97.574581)",30.345284,-97.574581,10200-10257 E Us 290 Hwy Svrd Eb,ACTIVE,2023-08-17 21:25:02
337719,54C1792C99A96A96A2881600F0CCE1D81061E8B8_16923...,2023-08-17 21:23:25,Crash Service,"(30.286561,-97.742754)",30.286561,-97.742754,W 23rd St / San Antonio St,ACTIVE,2023-08-17 21:25:02


In [10]:
# Create info box that will show up on map
info_box_template = """
<dl>
<dt>Issue</dt><dd>{Issue Reported}</dd>
<dt>Date & Time</dt><dd>{Published Date}</dd>
</dl>
"""

info = [info_box_template.format(**row) for index, row in df_sample.iterrows()]

In [11]:
locations = df_sample[['Latitude', 'Longitude']]
issues = df_sample['Issue Reported']

fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=info)

fig.add_layer(marker_layer)

# Display the figure
display(fig)


Figure(layout=FigureLayout(height='420px'))

In [12]:

fig = gmaps.figure()
display(fig)

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Get data from each row, add it to formatting template, and store in a list
inc_info = [info_box_template.format(**row) for index, row in df_sample.iterrows()]